# System Identification is used to Identify Hydrodynamic derivatives of Abkowitz Maneuver Model by ϵ Support Vector Regression
---
**Standard maneuvers, 35° turning circle, 10°/10°and 20°/20° zigzags, are simulated and compared with the predicted model by ϵ-SVR.**
**The predicted results are compared with results obtained from Planar Motion Mechanism (PMM) test.**

* This CODE is part of Research done at IITM under the Guidance of **Dr. Abhilash Sharma Somayajula** during July 2019- July 2020 and submitted by **Sudhanshu Shekhar Kanha** as MS Scholar in Department of Ocean Engineering.
**All rights Reserved**


### Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import tensorflow as tf

## Data

In [2]:
df=pd.read_csv('Mariner_10_10_zigzag_half_samples(8).csv')

In [ ]:
columns1= ['del_u','v','r','U','delta']
df=pd.DataFrame(data=df,columns=columns1)
df

In [4]:
del_u1=df['del_u']
v1=df['v']
r1=df['r']
U1=df['U']
delta1=df['delta']

In [5]:
len(del_u1)

251

In [6]:
del_u1[0]

0.000382035

## Parameters of Mariner Class Vessel

In [7]:
'''
Main Data parameter of Mariner class vessel

length overal (m)=Loa
length between perpendiculars (m)= Lpp
maximum beam (m)=B
design draft (m)=T
design displacement (m*3)= D
design speed (knots)=Uo
maximum rudder rate (degree/second)= delta_max
dimensionless mass of ship=m1
dimensionless moment of inertia=Iz1
dimensionless longitudinal coordinate of ships centre of gravity = xG1

'''
Loa=171.8
Lpp=160.93
L=Lpp
B=23.17
T=8.23
D=18541
Uo=7.9738
delta_max=5
m1=7.98*10**(-3)
Iz1=3.92*10**(-4)
xG1=-2.3*10**(-2)

'''
Predetermined hydrodynamic derivatives

Xudot1, Yvdot1 ,Yrdot1 , Nvdot1 , Nrdot1 

let, E= 10**(-5)

'''
E= 10**(-5)
Xudot1=(-840)*E
Yvdot1=-1546*E
Yrdot1=9*E
Nvdot1=23*E
Nrdot1=-83*E

In [8]:
'''
acceleration term: udot ,vdot, rdot
timestep =1 second
'''

def dot1(e):
    A=[]
    h=4
    for i in range(len(e)):
        e[-1]=0
        a=(e[i]-e[i-1])/h
        A.append(a)
    return A



In [9]:
udot1=dot1(del_u1)
vdot1=dot1(v1)
rdot1=dot1(r1)

In [10]:
len(udot1)

251

In [11]:
del_u1=df['del_u']
v1=df['v']
r1=df['r']
U1=df['U']
delta1=df['delta']

In [12]:
def original(del_u1):
    del_u1=del_u1[:len(del_u1)-1]
    return del_u1

In [13]:
del_u1=original(del_u1)
v1=original(v1)
r1=original(r1)

In [14]:
U1

0      7.974188
1      7.974252
2      7.973845
3      7.971037
4      7.967070
         ...   
246    7.636948
247    7.642178
248    7.647621
249    7.655741
250    7.662993
Name: U, Length: 251, dtype: float64

## Dimensional State Vectors

In [15]:
'''
Dimensional state vectors
'''

def A1(del_u1,v1,r1,U1,RA1):
    surge1=[]
    for i in range(len(del_u1)):
        a11=del_u1[i]*U1[i]
        a12=del_u1[i]*del_u1[i]
        a13=((del_u1[i])**3)/U1[i]
        a14=(v1[i])**2
        a15=((r1[i])**2)*L*L
        a16=v1[i]*r1[i]*L
        a17=((RA1[i])**2)*((U1[i])**2)
        a18=del_u1[i]*(RA1[i]**2)*U1[i]
        a19=v1[i]*RA1[i]*U1[i]
        a10=del_u1[i]*v1[i]*RA1[i]
        temp1=[a11,a12,a13,a14,a15,a16,a17,a18,a19,a10]
        surge1.append(temp1)
        
    return surge1 


def B1(del_u1,v1,r1,U1,RA1):
    sway1=[]
    for i in range(len(del_u1)):
        b11=U1[i]**2
        b12=del_u1[i]*U1[i]
        b13=((del_u1[i])**2)
        b14=(v1[i])*U1[i]
        b15=((r1[i]))*U1[i]*L
        b16=(v1[i]**3)/U1[i]
        b17=(((v1[i]**2)*r1[i]*L)/U1[i])
        b18=v1[i]*del_u1[i]
        b19=r1[i]*del_u1[i]*L
        b110=RA1[i]*(U1[i]**2)
        b111=(RA1[i]**3)*(U1[i]**2)
        b112=del_u1[i]*RA1[i]*U1[i]
        b113=(del_u1[i]**2)*RA1[i]
        b114=v1[i]*(RA1[i]**2)*U1[i]
        b115=(v1[i]**2)*RA1[i]
        
        temp1=[b11,b12,b13,b14,b15,b16,b17,b18,b19,b110,b111,b112,b113,b114,b115]
        sway1.append(temp1)
        
    return sway1


def C1(del_u1,v1,r1,U1,RA1):
    yaw1=[]
    for i in range(len(del_u1)):
        c11=U1[i]**2
        c12=del_u1[i]*U1[i]
        c13=((del_u1[i])**2)
        c14=(v1[i])*U1[i]
        c15=((r1[i]))*U1[i]*L
        c16=(v1[i]**3)/U1[i]
        c17=(((v1[i]**2)*r1[i]*L)/U1[i])
        c18=v1[i]*del_u1[i]
        c19=r1[i]*del_u1[i]*L
        c110=RA1[i]*(U1[i]**2)
        c111=(RA1[i]**3)*(U1[i]**2)
        c112=del_u1[i]*RA1[i]*U1[i]
        c113=(del_u1[i]**2)*RA1[i]
        c114=v1[i]*(RA1[i]**2)*U1[i]
        c115=(v1[i]**2)*RA1[i]
        
        temp1=[c11,c12,c13,c14,c15,c16,c17,c18,c19,c110,c111,c112,c113,c114,c115]
        yaw1.append(temp1)
        
    return yaw1

In [16]:
surge1=A1(del_u1,v1,r1,U1,delta1)
sway1=B1(del_u1,v1,r1,U1,delta1)
yaw1=C1(del_u1,v1,r1,U1,delta1)

In [17]:
surge1[0]

[0.00304641902871864,
 1.4595074122500003e-07,
 6.992346969775406e-12,
 0.0013194256385622009,
 1.0538751521908526e-05,
 -0.00011791988363479047,
 0.0,
 0.0,
 0.0,
 0.0]

In [18]:
surge1[1]

[0.0036027033630443676,
 2.0411601126399996e-07,
 1.1564467527828566e-11,
 0.0014124591889592038,
 0.02519687535750621,
 -0.005965698461351309,
 1.9370225254426787,
 0.00010974462001994427,
 -0.052306455292654604,
 -2.963492672533948e-06]

In [19]:
len(sway1)

251

In [20]:
surge1=np.reshape((surge1),(-1,10))
sway1=np.reshape((sway1),(-1,15))
yaw1=np.reshape((yaw1),(-1,15))

In [21]:
sway1.shape

(251, 15)

In [22]:
yaw1.shape

(251, 15)

In [23]:
surge1.shape

(251, 10)

## Input and output preparation for SVM Model-- Feature Engineering

In [24]:
'''
Input and output preparation for SVM Model
'''
def input_for_svm(del_u1,v1,r1,U1,RA1):
    
    Y_surge1=[]
   
    Y_sway1=[]
   
    Y_yaw1=[]
  
    
    for i in range(len(udot1)):
       
        tempa1=L*(m1-Xudot1)*(udot1[i])
      
        Y_surge1.append(tempa1)
       
        
        
        t21=L*(m1-Yvdot1)
        t221=vdot1[i]
        
        t23=L*L*(m1*xG1-Yrdot1)
        t241=rdot1[i]
        
        tempb1=(t21*t221)+(t23*t241)
        
        Y_sway1.append(tempb1)
        
        
        
        tempc21=(L*(m1*xG1-Nvdot1)*(vdot1[i]))+(L*L*(Iz1-Nrdot1)*(rdot1[i]))
        
        Y_yaw1.append(tempc21)
        
    return Y_surge1,Y_sway1,Y_yaw1


In [25]:
Y_surge1,Y_sway1,Y_yaw1=input_for_svm(del_u1,v1,r1,U1,delta1)

In [26]:
Y_sway1[0]

0.03429097234948236

In [27]:
len(Y_sway1)

251

In [28]:
Y_surge1=np.reshape((Y_surge1),(-1,1))
Y_sway1=np.reshape((Y_sway1),(-1,1))
Y_yaw1=np.reshape((Y_yaw1),(-1,1))

In [29]:
Y_surge1.shape

(251, 1)

In [30]:
Y_sway1[0]

array([0.03429097])

In [31]:
len(Y_sway1)

251

In [32]:
Y_sway1.shape

(251, 1)

## PMM Calculated Hydrodynamic Derivatives

In [33]:
'''
PMM calculated Hydrodynamic derivatives all is to be  multiplied by 10**(-5)
All hydrodynamic derivatives have zero dimension
'''
# E=10*(-5)

# X hydrodynamic derivatives

Xu=-184
Xuu=-110
Xuuu=-215
Xvv=-899
Xrr=18
Xdd=-95
Xddu=-190
Xvr=798
Xvd=93
Xvdu=93

X_original=[Xu,Xuu,Xuuu,Xvv,Xrr,Xdd,Xddu,Xvr,Xvd,Xvdu]

X_hydrodynamic_derivatives=['Xu','Xuu','Xuuu','Xvv','Xrr','Xdd','Xddu','Xvr','Xvd','Xvdu']


# Y hydrodynamic derivatives

Y0=-4
Yu=-8
Yuu=-4
Yv=-1160
Yr=-499
Yvvv=-8078
Yvvr=15356
Yvu=-1160
Yru=-499
Yd=278
Yddd=-90
Yud=556
Yuud=278
Yvdd=-4
Yvvd=1190

Y_original=[Y0,Yu,Yuu,Yv,Yr,Yvvv,Yvvr,Yvu,Yru,Yd,Yddd,Yud,Yuud,Yvdd,Yvvd]
Y_hydrodynamic_derivatives=['Y0','Yu','Yuu','Yv','Yr','Yvvv','Yvvr','Yvu','Yru','Yd','Yddd','Yud','Yuud','Yvdd','Yvvd']

# N hydrodynamic derivatives

N0=3
Nu=6
Nuu=3
Nv=-264
Nr=-166
Nvvv=-1636
Nvvr=-5483
Nvu=-264
Nru=-166
Nd=-139
Nddd=45
Nud=-278
Nuud=-139
Nvdd=13
Nvvd=-489

N_original=[N0,Nu,Nuu,Nv,Nr,Nvvv,Nvvr,Nvu,Nru,Nd,Nddd,Nud,Nuud,Nvdd,Nvvd]

N_hydrodynamic_derivatives=['N0','Nu','Nuu','Nv','Nr','Nvvv','Nvvr','Nvu','Nru','Nd','Nddd','Nud','Nuud','Nvdd','Nvvd']

## Saving the data after Feature Engineering

In [34]:
import pickle

pickle_out=open("surge1.pickle","wb")
pickle.dump(surge1,pickle_out)
pickle_out.close()

pickle_out=open("sway1.pickle","wb")
pickle.dump(sway1,pickle_out)
pickle_out.close()

pickle_out=open("yaw1.pickle","wb")
pickle.dump(yaw1,pickle_out)
pickle_out.close()

pickle_out=open("Y_surge1.pickle","wb")
pickle.dump(Y_surge1,pickle_out)
pickle_out.close()


pickle_out=open("Y_sway1.pickle","wb")
pickle.dump(Y_sway1,pickle_out)
pickle_out.close()

pickle_out=open("Y_yaw1.pickle","wb")
pickle.dump(Y_yaw1,pickle_out)
pickle_out.close()

In [35]:
import pickle
X_train=pickle.load(open("surge1.pickle","rb"))
y_train=pickle.load(open("sway1.pickle","rb"))
X_validate=pickle.load(open("yaw1.pickle","rb"))

y_validate=pickle.load(open("Y_surge1.pickle","rb"))
X_test=pickle.load(open("Y_sway1.pickle","rb"))
y_test=pickle.load(open("Y_yaw1.pickle","rb"))

## Making the Model

In [36]:
from sklearn.svm import LinearSVR
surge_model= LinearSVR(C=1e06,fit_intercept=False,dual=True,epsilon=1e-6,loss='squared_epsilon_insensitive',max_iter=10000,
                       random_state=None, tol=0.000001,verbose=0)
surge_model.fit(surge1,Y_surge1)

c:\users\user\anaconda3\envs\tfen\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\user\anaconda3\envs\tfen\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVR(C=1000000.0, dual=True, epsilon=1e-06, fit_intercept=False,
          intercept_scaling=1.0, loss='squared_epsilon_insensitive',
          max_iter=10000, random_state=None, tol=1e-06, verbose=0)

### Calculating X Hydrodynmaic Derivaties

In [37]:
b=surge_model.coef_
X_hydroderivatives=b*1e05
X_hydroderivatives

array([  105.56022262,  2497.79503635,   403.26194045, -5799.13053694,
        -247.24065375, -1875.05836453,   -85.42453829, -5950.58908937,
         -23.13602313,  3131.57648359])

In [38]:
surge_model.intercept_

0.0

In [39]:
sway_model= LinearSVR(C=1e06,fit_intercept=False,epsilon=1e-6,loss='squared_epsilon_insensitive',max_iter=10000,
                       random_state=None, tol=0.00001,verbose=0)
sway_model.fit(sway1,Y_sway1)

c:\users\user\anaconda3\envs\tfen\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\user\anaconda3\envs\tfen\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVR(C=1000000.0, dual=True, epsilon=1e-06, fit_intercept=False,
          intercept_scaling=1.0, loss='squared_epsilon_insensitive',
          max_iter=10000, random_state=None, tol=1e-05, verbose=0)

### Calculating Y Hydrodynmaic Derivaties

In [40]:
c=sway_model.coef_
Y_hydroderivatives=c*1e05

Y_hydroderivatives

array([ 1.23135441e-01,  9.94397766e+01,  1.78347611e+02, -1.77170350e+03,
       -8.11753382e+02, -2.99961557e+02,  3.93620040e+02,  2.28346408e+02,
       -6.96984832e+02, -1.41093543e+02, -4.29795508e+00, -5.79096614e+02,
        1.24754169e+02, -1.34249597e+02,  5.50095977e+02])

In [41]:
sway_model.intercept_

0.0

In [42]:
sway_model.n_iter_

10000

In [43]:
yaw_model= LinearSVR(C=1e06,fit_intercept=False,epsilon=1e-6,loss='squared_epsilon_insensitive',max_iter=10000,
                       random_state=None, tol=0.00001,verbose=0)
yaw_model.fit(sway1,Y_sway1)

c:\users\user\anaconda3\envs\tfen\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\user\anaconda3\envs\tfen\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVR(C=1000000.0, dual=True, epsilon=1e-06, fit_intercept=False,
          intercept_scaling=1.0, loss='squared_epsilon_insensitive',
          max_iter=10000, random_state=None, tol=1e-05, verbose=0)

### Calculating N Hydrodynmaic Derivaties

In [44]:
f=sway_model.coef_
N_hydroderivatives=f*1e05
N_hydroderivatives

array([ 1.23135441e-01,  9.94397766e+01,  1.78347611e+02, -1.77170350e+03,
       -8.11753382e+02, -2.99961557e+02,  3.93620040e+02,  2.28346408e+02,
       -6.96984832e+02, -1.41093543e+02, -4.29795508e+00, -5.79096614e+02,
        1.24754169e+02, -1.34249597e+02,  5.50095977e+02])

In [45]:
yaw_model.intercept_

0.0

## Comparing Predicted Hydrodynamic Derivatives & Experimentally Calculated by PMM test.

In [46]:
from tabulate import tabulate

Sr_No_X=range(1,len(X_original)+1)

table_X=zip(Sr_No_X,X_hydrodynamic_derivatives,X_original,X_hydroderivatives)
header_X=['Sr_No','X_hydrodynamic_derivatives','X_original','X_predicted']


print(tabulate(table_X,header_X))


Sr_No_Y=range(1,len(Y_original)+1)

table_Y=zip(Sr_No_Y,Y_hydrodynamic_derivatives,Y_original,Y_hydroderivatives)
header_Y=['Sr_No','Y_hydrodynamic_derivatives','Y_original','Y_predicted']


print(tabulate(table_Y,header_Y))


Sr_No_N=range(1,len(N_original)+1)

table_N=zip(Sr_No_N,N_hydrodynamic_derivatives,N_original,N_hydroderivatives)
header_N=['Sr_No','N_hydrodynamic_derivatives','N_original','N_predicted']


print(tabulate(table_N,header_N))


  Sr_No  X_hydrodynamic_derivatives      X_original    X_predicted
-------  ----------------------------  ------------  -------------
      1  Xu                                    -184       105.56
      2  Xuu                                   -110      2497.8
      3  Xuuu                                  -215       403.262
      4  Xvv                                   -899     -5799.13
      5  Xrr                                     18      -247.241
      6  Xdd                                    -95     -1875.06
      7  Xddu                                  -190       -85.4245
      8  Xvr                                    798     -5950.59
      9  Xvd                                     93       -23.136
     10  Xvdu                                    93      3131.58
  Sr_No  Y_hydrodynamic_derivatives      Y_original    Y_predicted
-------  ----------------------------  ------------  -------------
      1  Y0                                      -4       0.123135
      2  Yu

# Done!